### Normalization

Batch Normalization (BN), normalizes acrossmini-batch independently for each for each feature. -> (x1sample1+x1sample2+x1sample3)/3

Layer Normalization (LN) normalizes across the features independently for each for each sample. -> (x1+x2+x3+x4)/4


mean of normalization = 0 & variance of normalization = 1

In [ ]:
import torch
from torch import nn

class LayerNorm(nn.Module):
  def __init__(self, emb_dim):
    super().__init__()
    # Prevent a devison by 0
    self.eps = 1e-5
    # Undo the division of normalization
    self.scale = nn.Parameter(torch.ones(emb_dim))
    # Undo the subtraction of normalization
    self.shift = nn.Parameter(torch.zeros(emb_dim))

  def forward(self, x):
    # The last dimention (dim=-1) is usually features.
    mean = x.mean(dim=-1, keepdim=True)
    # unbiased sample statistic vs Population statistics
    var = x.var(dim=-1, keepdim=True, unbiased=True)
    norm_x = (x - mean) / torch.sqrt(var + self.eps)
    return self.scale * x + self.shift

### GELU Activation Function

In [ ]:
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1+ torch.tanh(
        torch.sqrt(torch.tensor(2.0 / torch.pi)) *
        (x + 0.44715 * torch.pow(x, 3))
    ))

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(config['emb_dim'], 4*config['emb_dim']),
        GELU(),
        nn.Linear(4*config['emb_dim'], config['emb_dim'])
    )

  def forward(self, x):
    return self.layers(x)

### Attention

In [ ]:
from torch import nn

class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, dropout, context_length, num_heads=2, qkv_bias=False):
    super().__init__()
    assert(d_out % num_heads == 0), "d_out must be divisible by num_heads"

    self.num_heads = num_heads
    self.d_out = d_out
    self.head_dim = d_out // num_heads

    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.out_proj = nn.Linear(d_out, d_out)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer(
        'mask',
        torch.triu(torch.ones(context_length, context_length), diagonal=1)
    )


  def forward(self, x):
    # x = batch, tokens, embedding dimention
    b, num_tokens, d_in = x.shape

    queries = self.W_query(x)
    keys = self.W_key(x)
    values = self.W_value(x)

    queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
    keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
    values = values.view(b, num_tokens, self.num_heads, self.head_dim)

    queries = queries.transpose(1, 2)
    keys = keys.transpose(1, 2)
    values = values.transpose(1, 2)

    attn_scores = queries @ keys.transpose(2, 3)

    mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
    attn_scores.masked_fill_(mask_bool, -torch.inf)

    attn_weights = torch.softmax(
        attn_scores / keys.shape[1]**0.5, dim=-1
    )
    attn_weights = self.dropout(attn_weights)
    context_vec = (attn_weights @ values).transpose(1, 2)

    context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
    context_vec = self.out_proj(context_vec) # optional projection

    return context_vec

### Transformers

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attn = MultiHeadAttention(d_in=config['emb_dim'],
                                       d_out=config['emb_dim'],
                                       dropout=config['dropout'],
                                       context_length=config['context_length'],
                                       num_heads=2,
                                       qkv_bias=False)
        self.norm1 = LayerNorm(emb_dim=config['emb_dim'])
        self.norm2 = LayerNorm(emb_dim=config['emb_dim'])
        self.ff = FeedForward(config)
        self.dropout = nn.Dropout(config['dropout'])


    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.attn(x)
        x = self.dropout(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.dropout(x)
        x = x + shortcut

        return x

### GPT Model

In [ ]:
import torch
import torch.nn as nn

class GPTModel(nn.Module):
    def __init__(self, config):
        super(GPTModel, self).__init__()
        self.config = config
        self.tok_emb = nn.Embedding(config['vocab_size'], config['emb_dim'])  # Token Embedding
        self.pos_emb = nn.Embedding(config['context_length'], config['emb_dim']) # Positional Embedding
        self.drop_emb = nn.Dropout(config['dropout'])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(config) for _ in range(config['n_layers'])])
        self.final_norm = LayerNorm(config['emb_dim'])
        self.output_head = nn.Linear(config['emb_dim'], config['vocab_size'])

    def forward(self, x):
        tok_emb = self.tok_emb(x)
        pos_emb = self.pos_emb(torch.arange(x.size(1)))
        x = tok_emb + pos_emb
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.output_head(x)
        # the outputs of the last linear layer
        return logits

In [ ]:
GPT_CONFIG_124M = {
    'vocab_size': 50257,  # Vocabulary size of gpt-2 tokenizer
    'context_length': 1024,
    'emb_dim': 768,
    'n_heads': 12,  # Number of attention heads
    'n_layers': 12, # Number of transformer blocks
    'dropout': 0.1,
    'qkv_bias': False # Query-Key-Value bias
}

In [ ]:
!pip install tiktoken==0.9.0

In [ ]:
import tiktoken

tokernizer = tiktoken.get_encoding('gpt2')

txt1 = 'Every effort moves you'
txt2 = 'Every day holds a'
batch = []

batch.append(torch.tensor(tokernizer.encode(txt1)))
batch.append(torch.tensor(tokernizer.encode(txt2)))
batch = torch.stack(batch)
batch

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

In [ ]:
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)

In [ ]:
sum([p.numel() for p in model.parameters()])

163059793

### Text Generation

In [ ]:
def generate_text():
  pass